# NYC Yellow Taxi Trip January and February 2023

## Q1. Downloading the data

In [3]:
!python -V

Python 3.9.19


In [4]:

import pandas as pd
import seaborn as sns
import numpy as np
import pyarrow.parquet as pq
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [5]:
# Open the Parquet file
parquet_file = pq.ParquetFile(r'C:\Users\sawsa\Documents\Data\GitHub_projects\MLOps_course2024\01-intro\Homework\data\yellow_tripdata_2023-01.parquet')

# Define an empty DataFrame or prepare to process batches individually
df = pd.DataFrame()

# Iterate over batches
for batch in parquet_file.iter_batches(batch_size=10000):  # Adjust batch_size as needed
    batch_df = batch.to_pandas()
    # Here you can process each batch_df as needed or concatenate them
    df = pd.concat([df, batch_df], ignore_index=True)

# Now df contains all the data loaded in chunks


In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### How many columns are there?

In [8]:
len(df)

3066766

In [9]:
# Using columns attribute
num_columns = len(df.columns)
print("Number of columns:", num_columns)

Number of columns: 19


## Q2. Computing duration

#### Now let's compute the duration variable. It should contain the duration of a ride in minutes.

### What's the standard deviation of the trips duration in January

In [10]:
# Compute the duration of each trip in seconds
df['duration_seconds'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()
# Convert duration to minutes
df['duration_minutes'] = df['duration_seconds'] / 60    

# Calculate standard deviation of trip durations in January
january_trips = df[df['tpep_pickup_datetime'].dt.month == 1]
std_dev_duration_january = january_trips['duration_minutes'].std()

print("Standard deviation of trip durations in January:", std_dev_duration_january)

Standard deviation of trip durations in January: 42.58564176427416


## Q3. Dropping outliers
#### Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [11]:
# Filter records where duration is between 1 and 60 minutes (inclusive)
filtered_df = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]

# # Optionally, you can also drop the temporary duration_seconds column
# filtered_df.drop(columns=['duration_seconds'], inplace=True)

# Check the distribution of the filtered duration variable
print("Distribution of duration variable after removing outliers:")
filtered_df['duration_minutes'].describe()

Distribution of duration variable after removing outliers:


count    3.009173e+06
mean     1.420486e+01
std      9.939386e+00
min      1.000000e+00
25%      7.216667e+00
50%      1.155000e+01
75%      1.818333e+01
max      6.000000e+01
Name: duration_minutes, dtype: float64

In [12]:
# Calculate the number of records left after dropping outliers
len(filtered_df)

3009173

### What fraction of the records left after you dropped the outliers?

In [13]:
# Calculate the fraction of records left
print("Fraction of records left after dropping outliers:")
len(filtered_df) / len(df)

Fraction of records left after dropping outliers:


0.9812202822125979

## Q4. One-hot encoding

### Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

### What's the dimensionality of this matrix (number of columns)?

In [14]:
# Convert pickup and dropoff location IDs to strings

filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Apply one-hot encoding to pickup and dropoff location IDs
ohe_df = pd.get_dummies(filtered_df[['PULocationID', 'DOLocationID']], sparse=True)


C:\Users\sawsa\AppData\Local\Temp\ipykernel_24780\2950968588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
C:\Users\sawsa\AppData\Local\Temp\ipykernel_24780\2950968588.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)


In [15]:
# Convert DataFrame into a list of dictionaries
list_of_dicts = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [16]:
# Initialize DictVectorizer
vectorizer = DictVectorizer()

# Fit the vectorizer on the list of dictionaries
vectorizer.fit(list_of_dicts)

DictVectorizer()

In [17]:
# Get the feature matrix
feature_matrix = vectorizer.transform(list_of_dicts)

In [18]:
print("Feature matrix shape:", feature_matrix.shape)

Feature matrix shape: (3009173, 515)


## Q5. Training a model
### Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

### What's the RMSE on train?

In [19]:
# Extract features from the feature matrix
X = feature_matrix
X

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [20]:
# Extract target variable (duration_minutes)
y = filtered_df['duration_minutes']
y

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration_minutes, Length: 3009173, dtype: float64

In [21]:
# Create a Linear Regression model
model = LinearRegression()

In [22]:
# Fit the model to the feature matrix and duration_minutes
model.fit(X,y)

LinearRegression()

In [23]:
# Predict durations using the trained model
y_pred = model.predict(X)

In [24]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("Root Mean Squared Error (RMSE) on training data:", rmse)

Root Mean Squared Error (RMSE) on training data: 7.649261937621321


## Q6. Evaluating the model
### Now let's apply this model to the validation dataset (February 2023).

In [25]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df


In [26]:
df_val = read_data(r'C:\Users\sawsa\Documents\Data\GitHub_projects\MLOps_course2024\01-intro\Homework\data\yellow_tripdata_2023-02.parquet')
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = vectorizer.transform(val_dicts)
y_val = df_val['duration'].values


In [27]:
y_pred_val = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print("Root Mean Squared Error (RMSE) on validation data:", rmse_val)

Root Mean Squared Error (RMSE) on validation data: 7.811817646307258
